In [ ]:
import numpy
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D ##library for 3d projection plots
%matplotlib inline

# High-Level Waste : Technology

Having covered the policy related to high level waste, we will now discuss the technology.

## Learning Objectives

At the end of this lesson, you will be equipped to:

- Recognize the relationship between decay and repository heat burden.
- Calculate the decay heat of UNF as a function of reactor power and time since discharge.
- Calculate radiotoxicity of a mixture of isotopes over time.
- List mechanisms of radionuclide contaminant transport.
- Differentiate between reducing and oxidizing geologic host media. 
- List the key benefits and challenges in reducing geologic host media.
- List the key benefits and challenges in oxidizing geologic host media.



## Radioactivity

Recall: The SI unit of activity is the becquerel (Bq), equal to one reciprocal second.

\begin{align}
A(i) &= -\frac{dN_i}{dt}\\
&= \lambda_iN_i
\end{align}

And, given these decays, we also know that 

\begin{align}
N(t) = N_0e^{t/\tau}
\end{align}

## Decay Heat

How do we get from radiation to heat?

![https://upload.wikimedia.org/wikipedia/commons/thumb/5/55/Question_Mark.svg/2000px-Question_Mark.svg.png](https://upload.wikimedia.org/wikipedia/commons/thumb/5/55/Question_Mark.svg/2000px-Question_Mark.svg.png)


\begin{align}
f(t) &= \sum_i\left( \bar{E}_{\beta,i} + \bar{E}_{\gamma, i} + \bar{E}_{\alpha, i}\right)A(i)\\
     &= \sum_i\left( \bar{E}_{\beta,i} + \bar{E}_{\gamma, i} + \bar{E}_{\alpha, i}\right)\lambda_iN_i(t)\\
\end{align}


Recall that we know how to get the populations $N_i$ of all isotopes built up in the reactor. 




In [ ]:
from pyne import data

data.decay_const('u238')
 
from pyne.material import Material
lwr_snf_ma = Material({'U234': 0.12, 
                    'U236': 4.18,
                    'Np237': 0.75,
                    '236Pu': 9.2E-6,
                    '238Pu': 0.22,
                    '239Pu': 5.28,
                    '240Pu': 2.17,
                    '241Pu': 1.02,
                    '242Pu': 0.35,
                    '241Am': 0.05,
                    '243Am': 0.09,
                    '242Cm': 4.9E-3,
                    '244Cm': 3.3E-2},
                   1000)

lwr_snf_ma.decay(60*60*24*365*100).comp

## Decay Heat and Geologic Heat Capacity

![concept_features.png](./concept_features.png)


![heat_lims.png](./heat_lims.png)

## Toxicity

One possibility is to use toxicity as a classifying metric. It captures the 

\begin{align}
\mbox{Toxicity}(i,k) &= \frac{A(i)}{DAC(i,k)}\\
i &= \mbox{isotope}\\
k &= \mbox{medium index, air or water}\\
DAC &= \mbox{derived air concentration of isotope i in air or water.} 
\end{align}

Warning : DAC is confusing. It generally is reported in $\frac{\mu Ci}{mL}$. From the NRC glossary, the definition is:

> Derived air concentration (DAC) The concentration of a given radionuclide in air which, if breathed by the reference man for a working year of 2,000 hours under conditions of light work (with an inhalation rate of 1.2 cubic meters of air per hour), results in an intake of one annual limit on intake (ALI).

Ok, great, so what is ALI?

> ALI is the smaller value of intake of a given radionuclide in a year by the "reference man" that would result in a committed effective dose equivalent (CEDE) of 5 rems (0.05 sievert) or a committed dose equivalent (CDE) of 50 rems (0.5 sievert) to any individual organ or tissue. 

## Empirical Activity Fits

\begin{align}
R(t) &=
\begin{cases}
 A_1e^{\left(\frac{1}{A_2 + A_3t}\right)} & t<30y\\
 B_1t^{-a}\left[1+ \frac{B_2}{1+\left(\frac{t}{B_3}\right)^4}\right] & 30<t<1\times10^5y\\
 \end{cases}\\
\end{align}
For a 1250 Mwe PWR with $\eta=0.33$, average specific power of 37.5 MWth/tHM, and 33,000 MWd/tHM burnup, the coefficients are:

\begin{align}
A_1 &= 1.42\times10^5\\
 A_2 &= 0.296\\
 A_3 &= 7.22\times10^{-2}\\
 a &= 0.2680\\
 B_1 &= 1.98\times10^5\\
 B_2 &= 10\\
 B_3 &= 88\\
\end{align}

## Diffusion
[Diffusion code below from 12 steps to Navier Stokes, Lorena Barba.](http://nbviewer.jupyter.org/github/barbagroup/CFDPython/blob/master/lessons/09_Step_7.ipynb)

We begin with the 2D-diffusion equation:

$$\frac{\partial u}{\partial t} = \nu \frac{\partial ^2 u}{\partial x^2} + \nu \frac{\partial ^2 u}{\partial y^2}$$

Without going into details, just note that it is possible to discretize second order derivatives. The following scheme uses a _forward difference_ in time and two second-order derivatives. 

$$\frac{u_{i,j}^{n+1} - u_{i,j}^n}{\Delta t} = \nu \frac{u_{i+1,j}^n - 2 u_{i,j}^n + u_{i-1,j}^n}{\Delta x^2} + \nu \frac{u_{i,j+1}^n-2 u_{i,j}^n + u_{i,j-1}^n}{\Delta y^2}$$

This method requires that you reorganize the discretized equation and solve for $u_{i,j}^{n+1}$
\begin{align}
u_{i,j}^{n+1} = u_{i,j}^n &+ \frac{\nu \Delta t}{\Delta x^2}(u_{i+1,j}^n - 2 u_{i,j}^n + u_{i-1,j}^n) \\
&+ \frac{\nu \Delta t}{\Delta y^2}(u_{i,j+1}^n-2 u_{i,j}^n + u_{i,j-1}^n)
\end{align}

In [1]:
### variable declarations
nx = 31
ny = 31
nt = 17
nu = 0.05
dx = 2 / (nx - 1)
dy = 2 / (ny - 1)
sigma = .25
dt = sigma * dx * dy / nu

x = numpy.linspace(0, 2, nx)
y = numpy.linspace(0, 2, ny)

u = numpy.ones((ny, nx))  # create a 1xn vector of 1's
un = numpy.ones((ny, nx))

### Assign initial conditions
# set hat function I.C. : u(.5<=x<=1 && .5<=y<=1 ) is 2
u[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2  

fig = pyplot.figure()
ax = fig.gca(projection='3d')
X, Y = numpy.meshgrid(x, y)
surf = ax.plot_surface(X, Y, u, rstride=1, cstride=1, cmap=cm.viridis,
        linewidth=0, antialiased=False)

ax.set_xlim(0, 2)
ax.set_ylim(0, 2)
ax.set_zlim(1, 2.5)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$');

NameError: name 'numpy' is not defined

\begin{align}
u_{i,j}^{n+1} = u_{i,j}^n &+ \frac{\nu \Delta t}{\Delta x^2}(u_{i+1,j}^n - 2 u_{i,j}^n + u_{i-1,j}^n) \\
&+ \frac{\nu \Delta t}{\Delta y^2}(u_{i,j+1}^n-2 u_{i,j}^n + u_{i,j-1}^n)
\end{align}

In [ ]:
###Run through nt timesteps
def diffuse(nt):
    u[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2  
    
    for n in range(nt + 1): 
        un = u.copy()
        u[1:-1, 1:-1] = (un[1:-1,1:-1] + 
                        nu * dt / dx**2 * 
                        (un[1:-1, 2:] - 2 * un[1:-1, 1:-1] + un[1:-1, 0:-2]) +
                        nu * dt / dy**2 * 
                        (un[2:,1: -1] - 2 * un[1:-1, 1:-1] + un[0:-2, 1:-1]))
        u[0, :] = 1
        u[-1, :] = 1
        u[:, 0] = 1
        u[:, -1] = 1

    
    fig = pyplot.figure()
    ax = fig.gca(projection='3d')
    surf = ax.plot_surface(X, Y, u[:], rstride=1, cstride=1, cmap=cm.viridis,
        linewidth=0, antialiased=True)
    ax.set_zlim(1, 2.5)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$');
    

In [ ]:
diffuse(0)

In [ ]:
diffuse(50)

## Wrap up


At the end of this lesson, you should know:
    
- Energy released from decay is deposited in surrounding geologic host media.
- Both analytical and numerical methods for determining decay heat contribution of an isotope.
- Calculate radiotoxicity of a mixture of isotopes over time.
- List mechanisms of radionuclide contaminant transport.
- Differentiate between reducing and oxidizing geologic host media. 
- List the key benefits and challenges in reducing geologic host media.
- List the key benefits and challenges in oxidizing geologic host media.

